# Lab 3-1: Data Drift Detection

## Part 1: Drift 감지 기초

이 노트북에서는 **Data Drift**의 개념을 이해하고, **KS Test**를 사용하여 Drift를 감지하는 방법을 학습합니다.

### 학습 목표
- Data Drift의 개념 이해
- KS Test (Kolmogorov-Smirnov Test) 수행
- Drift Score 계산 및 해석

### 소요 시간: 약 30분

---

## 📚 Data Drift란?

**Data Drift**는 프로덕션 환경에서 모델에 입력되는 데이터의 분포가 학습 시 사용했던 데이터의 분포와 달라지는 현상입니다.

### Drift의 원인
- 사용자 행동 패턴 변화
- 시장/환경 변화
- 계절적 요인
- 데이터 수집 오류

### Drift의 영향
- 모델 성능 저하
- 예측 정확도 감소
- 비즈니스 지표 악화

### Drift 감지 방법
| 방법 | 설명 |
|------|------|
| **KS Test** | 두 분포가 같은지 통계적으로 검정 |
| **Chi-Square Test** | 범주형 데이터의 분포 비교 |
| **PSI (Population Stability Index)** | 분포 변화 정도 측정 |

---

## Step 1: 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from scipy.stats import ks_2samp

print("Libraries imported successfully!")

## Step 2: 데이터 로드

**California Housing** 데이터셋을 사용합니다.

### 데이터셋 특성
| Feature | 설명 |
|---------|------|
| MedInc | 중위 소득 |
| HouseAge | 주택 연령 |
| AveRooms | 평균 방 수 |
| AveBedrms | 평균 침실 수 |
| Population | 인구 |
| AveOccup | 평균 거주자 수 |
| Latitude | 위도 |
| Longitude | 경도 |
| MedHouseVal | 중위 주택 가격 (타겟) |

In [ ]:
# Load California Housing dataset
data = fetch_california_housing(as_frame=True)
df = data.frame

print(f"Dataset shape: {df.shape}")
print(f"Features: {list(df.columns)}")
df.head()

## Step 3: Reference Data와 Current Data 준비

### 시나리오
- **Reference Data**: 모델 학습에 사용된 과거 데이터 (5000 샘플)
- **Current Data**: 현재 프로덕션에서 수집된 데이터 (3000 샘플)

### Drift 시뮬레이션
Current Data의 `MedInc` (중위 소득) feature에 인위적인 변화를 줍니다:
- 값을 1.5배 증가
- 노이즈 추가

In [ ]:
# Reference data (baseline - historical data)
reference_data = df.sample(n=5000, random_state=42)
print(f"Reference data: {len(reference_data)} samples")

# Current data (production data)
current_data = df.sample(n=3000, random_state=123)
current_data = current_data.copy()

# Simulate drift on MedInc feature (increase by 50% + noise)
current_data['MedInc'] = current_data['MedInc'] * 1.5 + np.random.normal(0, 0.3, len(current_data))
print(f"Current data: {len(current_data)} samples (with simulated drift on MedInc)")

## Step 4: 분포 시각화

Reference Data와 Current Data의 분포를 비교합니다.

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

features_to_plot = ['MedInc', 'HouseAge', 'AveRooms', 'Population']

for ax, feature in zip(axes.flatten(), features_to_plot):
    ax.hist(reference_data[feature], bins=30, alpha=0.5, label='Reference', density=True)
    ax.hist(current_data[feature], bins=30, alpha=0.5, label='Current', density=True)
    ax.set_title(f'{feature} Distribution')
    ax.legend()

plt.tight_layout()
plt.show()

print("Notice: MedInc shows clear distribution shift (drift)!")

---

## Step 5: KS Test 수행

### KS Test (Kolmogorov-Smirnov Test)란?

두 샘플이 **동일한 분포**에서 왔는지 검정하는 비모수적(non-parametric) 통계 방법입니다.

### 해석
- **p-value < 0.05**: 두 분포가 통계적으로 유의미하게 다름 → **Drift 감지!**
- **p-value >= 0.05**: 두 분포가 유의미하게 다르다고 할 수 없음 → No Drift

### 코드
```python
from scipy.stats import ks_2samp

statistic, p_value = ks_2samp(reference_data, current_data)
drift_detected = p_value < 0.05
```

In [ ]:
# Configuration
SIGNIFICANCE_LEVEL = 0.05  # p-value threshold

print("Performing KS Test for each feature...")
print("=" * 60)

drift_results = []

for col in reference_data.columns:
    # KS Test
    statistic, p_value = ks_2samp(reference_data[col], current_data[col])
    
    # p < 0.05 means drift detected
    drift_detected = p_value < SIGNIFICANCE_LEVEL
    
    drift_results.append({
        'feature': col,
        'ks_statistic': statistic,
        'p_value': p_value,
        'drift': drift_detected
    })
    
    status = "YES" if drift_detected else "NO "
    print(f"Feature: {col:12s} | Drift: {status} | p-value: {p_value:.4f} | KS stat: {statistic:.4f}")

print("=" * 60)

---

## Step 6: Drift Score 계산

### Drift Score 공식
```
Drift Score = (Drift가 감지된 Feature 수) / (전체 Feature 수)
```

### 해석
- **0.0**: 모든 feature가 안정적 (No Drift)
- **0.3 (30%)**: Threshold - 이 이상이면 재학습 고려
- **1.0**: 모든 feature에서 Drift 발생

In [ ]:
# Calculate Drift Score
drift_df = pd.DataFrame(drift_results)

n_drifted = drift_df['drift'].sum()
n_total = len(drift_df)
drift_score = n_drifted / n_total

# Threshold
DRIFT_THRESHOLD = 0.3  # 30%

print("=" * 60)
print("  Drift Summary")
print("=" * 60)
print(f"  Drifted Features: {n_drifted}/{n_total}")
print(f"  Drift Score: {drift_score:.2f} ({drift_score*100:.0f}%)")
print(f"  Threshold: {DRIFT_THRESHOLD:.2f} ({DRIFT_THRESHOLD*100:.0f}%)")
print()

if drift_score > DRIFT_THRESHOLD:
    print("  Status: DRIFT DETECTED! Consider model retraining.")
else:
    print("  Status: No significant drift. Model is stable.")

print("=" * 60)

## Step 7: 결과 테이블 확인

In [ ]:
# Display results as DataFrame
drift_df['drift_status'] = drift_df['drift'].apply(lambda x: 'DRIFT' if x else 'OK')
drift_df = drift_df.sort_values('p_value')

print("Drift Detection Results (sorted by p-value):")
drift_df[['feature', 'ks_statistic', 'p_value', 'drift_status']]

---

## ✅ Part 1 완료 체크리스트

- [ ] Data Drift 개념 이해
- [ ] Reference Data vs Current Data 구분
- [ ] KS Test 수행 및 p-value 해석
- [ ] Drift Score 계산 (예: 0.11 = 11%)
- [ ] Threshold 기반 Drift 판정

---

## 📚 핵심 개념 정리

### 1. KS Test
```python
from scipy.stats import ks_2samp

statistic, p_value = ks_2samp(reference, current)
drift_detected = p_value < 0.05
```

### 2. Drift Score
```python
drift_score = n_drifted / n_total

if drift_score > 0.3:  # 30% threshold
    trigger_retraining()
```

### 3. 다음 단계
- Part 2에서는 이 Drift Detection을 **Kubeflow Pipeline**으로 자동화합니다.

---

## 🔜 다음 단계

**Part 2: Monitoring Pipeline** (`2_monitor_pipeline.ipynb`)에서는:
- Kubeflow Pipeline으로 Drift 모니터링 자동화
- MLflow에 메트릭 기록
- Alert 시스템 구축